# 7 - Multi-Agent
- This example code utilizes the Crew AI framework to demonstrate multi-agent collaboration using Google Gemini. 
- It defines two distinct agents: a 'Senior Research Analyst' tasked with finding and summarizing AI trends, and a 'Technical Content Writer' responsible for creating a blog post. 
- Two tasks are set up, with the research task assigned to the researcher and the writing task assigned to the writer. Crucially, the writing task is configured to use the output of the research task as its context, establishing a sequential workflow. 
- A Crew is then instantiated with these agents and tasks, specifying a sequential process. Finally, crew.kickoff() executes the defined workflow, allowing the agents to collaborate by passing information from research to writing to produce the final blog post.

- Note that Google models failed and OpenAI owrked as per this providers list - https://docs.litellm.ai/docs/providers


In [11]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Verify if API key is present
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("OpenAI API key not found. Please check your .env file.")

# Define the language model to use (optional, defaults to env var)

# If using OpenAI (default in CrewAI if OPENAI_API_KEY is set):
llm = ChatOpenAI(model="gpt-4o-mini") # Or other suitable mode

In [12]:
# Define Agents with specific roles and goals

# Agent 1: Researcher
researcher = Agent(
    role='Senior Research Analyst',
    goal='Find and summarize the latest trends in AI.',
    backstory="You are an experienced research analyst with a knack for identifying key trends and synthesizing information.",
    verbose=True,
    allow_delegation=False, # This agent focuses on its own research
    llm=llm # Using the defined Gemini LLM
)

# Agent 2: Writer
writer = Agent(
    role='Technical Content Writer',
    goal='Write a clear and engaging blog post based on research findings.',
    backstory="You are a skilled writer who can translate complex technical topics into accessible content.",
    verbose=True,
    allow_delegation=False, # This agent focuses on writing
    llm=llm # Using the defined Gemini LLM
)

In [13]:
# Define Tasks for the agents

# Task 1: Research Task (assigned to researcher)
research_task = Task(
    description="Research the top 3 emerging trends in Artificial Intelligence in 2025. Focus on practical applications and potential impact.",
    expected_output="A detailed summary of the top 3 AI trends, including key points and sources (if applicable).",
    agent=researcher, # Assign this task to the researcher agent
)

# Task 2: Writing Task (assigned to writer)
# This task depends on the output of the research_task
writing_task = Task(
    description="Write a 200-word blog post based on the research findings provided by the researcher. The post should be engaging and easy to understand for a general audience.",
    expected_output="A complete 500-word blog post about the latest AI trends.",
    agent=writer, # Assign this task to the writer agent
    context=[research_task] # The writer needs the output of the research task
)

In [14]:
# Create the Crew with both agents and tasks
# Process.sequential means tasks are executed in the order they appear in the tasks list
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,
    llm=llm # Using the defined Gemini LLM at the crew level as well
)

In [15]:
# Execute the Crew
print("## Running the multi-agent collaboration task ##")
result = crew.kickoff()

print("\n## Crew Output ##")
print(result)

## Running the multi-agent collaboration task ##
# Agent: Senior Research Analyst
## Task: Research the top 3 emerging trends in Artificial Intelligence in 2025. Focus on practical applications and potential impact.


# Agent: Senior Research Analyst
## Final Answer: 
In 2025, the landscape of Artificial Intelligence is poised for transformative changes across various sectors. Here are the top three emerging trends, highlighting their practical applications and potential impacts:

1. **Generative AI Advancements**  
   Generative AI has evolved significantly, moving beyond text and image generation to encompass interactive and multi-modal systems. Leveraging deep learning, especially in transformer models, organizations can create comprehensive content - from videos to audio - with minimal human intervention. For example, in creative industries, companies like OpenAI and Google Brain are developing tools that allow filmmakers and music producers to auto-generate scripts or melodies bas